In [1]:
from bagg import bagginTextModels as btm
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing  import  Normalizer
import numpy as np
from flair.models import TextClassifier
from flair.data import Sentence
import pickle
import torch

In [2]:
import json
import pandas as pd

In [3]:
ml='models/malayalam_params.json'
mp=json.load(open(ml))
mlf='data/malayalam_train.json'
data=pd.read_json(mlf, lines=True)
mX=data.text.values
mY=data.klass.values
mdata=pd.read_json('data/malayalam_dev.json', lines=True)
xmm=[txt for txt in mdata.text.values]
ymm=[txt for txt in mdata.klass.values]
mtdata=pd.read_json('data/malayalam_test.json')
xtmm=[txt for txt in mtdata.text.values]
#ytmm=[txt for txt in mtdata.klass.values]
idmm=[txt for txt in mtdata.id.values]

In [4]:
test_malayalam=pd.read_csv('data/malayalam_test_answer.tsv',sep='\t')

In [5]:
test_malayalam.head()

,id,text,category
0,ml_sen_1,Bollywood film Newton inte remake aano?,unknown_state
1,ml_sen_2,endukond viewrs koodunnilla ?? ippozhum 2.8m a...,unknown_state
2,ml_sen_3,Mara paazhu mega mairananil ninnum ethil koodu...,Negative
3,ml_sen_4,Video nay cang xem cang thit,not-malayalam
4,ml_sen_5,Sunny chechiye kaanan vannathu njan maathram aano,unknown_state


In [6]:
test_malayalam.groupby('category').count()

,id,text
category,,
Mixed_feelings,70,70
Negative,138,138
Positive,565,565
not-malayalam,177,177
unknown_state,398,398


In [7]:
tl='models/tamil_params.json'
tp=json.load(open(tl))
tlf='data/tamil_train.json'
data=pd.read_json(tlf, lines=True)
tX=data.text.values
tY=data.klass.values
tdata=pd.read_json('data/tamil_dev.json', lines=True)
xtt=[txt for txt in tdata.text.values]
ytt=[txt for txt in tdata.klass.values]
ttdata=pd.read_json('data/tamil_test.json')
xttt=[txt for txt in ttdata.text.values]
#ytmm=[txt for txt in mtdata.klass.values]
idtt=[txt for txt in ttdata.id.values]

In [40]:
tdata.groupby('category').count()

,text,klass
category,,
Mixed_feelings,141,141
Negative,165,165
Positive,857,857
not-Tamil,29,29
unknown_state,68,68


In [8]:
test_tamil=pd.read_csv('data/tamil_test_answer.tsv',sep='\t')
test_tamil.groupby('category').count()

,id,text
category,,
Mixed_feelings,377,377
Negative,424,424
Positive,2075,2075
not-Tamil,100,100
unknown_state,173,173


In [9]:
bm=pickle.load(open('malayalam_model_test.pk','rb'))

In [10]:
bm.sol

     fun: -0.7021801669640152
 message: 'Optimization terminated successfully.'
    nfev: 456
     nit: 5
 success: True
       x: array([0.9472041 , 0.70713477, 0.12899314, 0.31150246, 0.10758394])

In [12]:
bt=pickle.load(open('tamil_model_test.pk','rb'))

In [13]:
bt.sol

     fun: -0.6276731092215135
 message: 'Optimization terminated successfully.'
    nfev: 156
     nit: 1
 success: True
       x: array([0.70178318, 0.01102967, 0.13731137, 0.48926635, 0.31429712])

In [14]:
bm.models

[{'name': 'mtc',
  'xv': <540x10880 sparse matrix of type '<class 'numpy.float64'>'
  	with 3466 stored elements in Compressed Sparse Row format>,
  'xt': <4851x10880 sparse matrix of type '<class 'numpy.float64'>'
  	with 38484 stored elements in Compressed Sparse Row format>,
  'clf': LinearSVC(max_iter=5000),
  'macroF1': 0.6023145368099656,
  'weightedF1': 0.6672294706402957,
  'probas': array([[-0.68201334, -0.52748629, -0.06507287, -0.33454369, -0.37478268],
         [-0.52138065, -0.4793826 , -0.41920715,  0.37942332, -0.42267939],
         [-0.49801925, -0.46564894, -0.22763727,  0.22330143, -0.65838107],
         ...,
         [-0.59172242, -0.36718178,  0.26100673, -0.42836204, -0.51324806],
         [-0.46495199, -0.40552459,  0.43260881, -0.58865258, -0.29275795],
         [-0.54847075, -0.45493872,  0.31548873, -0.41247846, -0.47174037]])},
 {'name': 'charMultiEmb',
  'xv': array([[ 0.0370688 ,  0.01904859, -0.10015232, ...,  0.21213035,
          -0.18509467,  0.20295201]

In [170]:
clf=bm.models[0]['clf']
textModel=bm.textModels['mtc']

In [177]:
#with torch.cuda.device(1):
for m in bm.models:
    yp=bm.LabelEncoder.inverse_transform(np.argmax(m['probas'],axis=1))
    f1=f1_score(ymm,yp,average='weighted')
    precision=precision_score(ymm,yp,average='weighted')
    recall=recall_score(ymm,yp,average='weighted')
    print(m['name'],precision,recall,f1)
#with torch.cuda.device(0):
#torch.device("cuda")
#y=bm.LabelEncoder.inverse_transform(bm.predict(xmm))
#f1=f1_score(ymm,yp,average='weighted')
#precision=precision_score(ymm,yp,average='weighted')
#recall=recall_score(ymm,yp,average='weighted')
#print(m['name'],precision,recall,f1)

mtc 0.6704708773916827 0.6777777777777778 0.6672294706402957
charMultiEmb 0.647844900241023 0.6574074074074074 0.649643910930063
charLangMultiEmb 0.6318649287440391 0.6370370370370371 0.6317317635343345
langMultiEmb 0.6295819111490476 0.6370370370370371 0.6309798266252948
multiEmb 0.6381611027638446 0.6481481481481481 0.6406484165342327


In [180]:
bm.params

{'_accuracy': 0.7103092783505155,
 '_all_f1': {'Mixed_feelings': 0.45161290322580644,
  'Negative': 0.6486486486486487,
  'Positive': 0.7607497243660418,
  'not-malayalam': 0.7449392712550607,
  'unknown_state': 0.6732283464566928},
 '_all_precision': {'Mixed_feelings': 0.7777777777777778,
  'Negative': 0.7792207792207793,
  'Positive': 0.6818181818181818,
  'not-malayalam': 0.7540983606557377,
  'unknown_state': 0.7184873949579832},
 '_all_recall': {'Mixed_feelings': 0.3181818181818182,
  'Negative': 0.5555555555555556,
  'Positive': 0.8603491271820449,
  'not-malayalam': 0.736,
  'unknown_state': 0.6333333333333333},
 '_geometricf1': 0.11185204110260294,
 '_harmonicf1': 0.633158630830261,
 '_macrof1': 0.6558357787904502,
 '_macrof1accuracy': 0.465846238749093,
 '_macrorecall': 0.6206839668505504,
 '_microf1': 0.7103092783505155,
 '_score': 0.6558357787904502,
 '_time': 6.249446630477905,
 '_weightedf1': 0.7008352655545667,
 'del_diac': True,
 'del_dup': False,
 'del_punc': True,
 'em

In [15]:
bt.params

{'_accuracy': 0.7035730039700044,
 '_all_f1': {'Mixed_feelings': 0.15568862275449102,
  'Negative': 0.32346241457858765,
  'Positive': 0.8268510515701526,
  'not-Tamil': 0.6857142857142857,
  'unknown_state': 0.2},
 '_all_precision': {'Mixed_feelings': 0.30952380952380953,
  'Negative': 0.3901098901098901,
  'Positive': 0.7536764705882353,
  'not-Tamil': 0.7272727272727273,
  'unknown_state': 0.4838709677419355},
 '_all_recall': {'Mixed_feelings': 0.104,
  'Negative': 0.27626459143968873,
  'Positive': 0.9157626037013401,
  'not-Tamil': 0.6486486486486487,
  'unknown_state': 0.12605042016806722},
 '_geometricf1': 0.005720410898864342,
 '_harmonicf1': 0.2911304574531329,
 '_macrof1': 0.4383432749235034,
 '_macrof1accuracy': 0.3084064947079788,
 '_macrorecall': 0.4141452527915489,
 '_microf1': 0.7035730039700044,
 '_score': 0.4383432749235034,
 '_time': 8.466909170150757,
 '_weightedf1': 0.6582578081996499,
 'del_diac': True,
 'del_dup': True,
 'del_punc': True,
 'emo_option': 'group',
 

In [117]:
f1_score(ytt,yp,average='weighted')
precision=precision_score(ytt,yp,average='weighted')
recall=recall_score(ytt,yp,average='weighted')
print(m['name'],precision,recall,f1)

multiEmb 0.0 0.0 0.0


In [139]:
print(torch.cuda.current_device())

True

In [70]:
tamil_vs=textModel.transform(xttt)

In [72]:
y_tamil=bm.LabelEncoder.inverse_transform(clf.predict(tamil_vs))

In [44]:
ttdata['label']=y_tamil

In [51]:
[(m['name'],m['weightedF1']) for m in bm.models]

[('mtc', 0.6317354311693437),
 ('charMultiEmb', 0.5936641944409738),
 ('charLangMultiEmb', 0.6121624515987472),
 ('langMultiEmb', 0.6099756291039425),
 ('multiEmb', 0.5868472572202869)]

In [1]:
ypp=bm.LabelEncoder.inverse_transform(bm.predict(xttt))

ymt

In [86]:
tamil_vs

<3149x16319 sparse matrix of type '<class 'numpy.float64'>'
	with 22751 stored elements in Compressed Sparse Row format>

In [128]:
f1_score(test_tamil.category.values,ypp,average='weighted')

0.6018114606742359

In [45]:
ttdata.to_csv('UMSNH_tamil.tsv',index=None, sep='\t')

In [57]:
ttdata.iloc[3].text

'70 vayasulayum thanoda rasigargala sandhosapadutha ipdi ulaikuraru ivara poi kayapaduthuranungale echainga ulachu vaalanum'

In [56]:
ttdata

,id,text,label
0,ta_sent_1,Yarayellam FDFS ppga ippove ready agitinga,Positive
1,ta_sent_2,Ennada viswasam mersal sarkar madhri time la l...,Positive
2,ta_sent_3,yuvan vera level ya .... valuable script. SK i...,Positive
3,ta_sent_4,70 vayasulayum thanoda rasigargala sandhosapad...,Mixed_feelings
4,ta_sent_5,all the best anna...Telugu makkal selvan fans,Positive
...,...,...,...
3144,ta_sent_3145,Tamil krish ah irukum oh...,Positive
3145,ta_sent_3146,Thalaivaaaaaa... trailer ye pattaiya kelapudhe...,Positive
3146,ta_sent_3147,Innum neraya neraya neraya neraya neraya,Positive
3147,ta_sent_3148,1:05 to 1:30 Vere level masss,Positive


In [18]:
keys_map={'del_diac': 'diacritic-removal','del_dup': 'duplication-removal','del_punc': 'punctuation-removal',
 'emo_option': 'emojis-handler',
 'ent_option': 'entities-handler',
 'hashtag_option': 'hashtag-handlers',
 'lc': 'Lower Case',
 'num_option': 'number-handler',
 'select_conn': 'select\_conn',
 'select_ent': 'select\_ent',
 'select_suff': 'select_suff',
 'token_list': 'skip-grams/ngrams/qgrams',
 'token_max_filter': 'token\_max\_filter',
 'token_min_filter': 'token\_min\_filter',
 'url_option': 'url-handler',
 'usr_option': 'user-handler',
 'weighting': 'Weighting scheme'}

In [19]:
for k,v in keys_map.items():
    print(f"{keys_map[k]}&{bm.params[k]} & {bt.params[k]}")

diacritic-removal&True & True
duplication-removal&False & True
punctuation-removal&True & True
emojis-handler&none & group
entities-handler&none & none
hashtag-handlers&none & delete
Lower Case&True & False
number-handler&delete & none
select\_conn&False & False
select\_ent&False & False
select_suff&False & False
skip-grams/ngrams/qgrams&[-3, -2, -1, 2, 3, 5, 9] & [[2, 1], -3, -2, 1, 3]
token\_max\_filter&1.0 & 1.0
token\_min\_filter&-1 & -1
url-handler&group & group
user-handler&group & group
Weighting scheme&tfidf & tf
